In [ ]:
import numpy as np
import tensorflow 
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
print('Training data shape: ', train_images.shape, train_labels.shape)
print('Testing data shape: ', test_images.shape, test_labels.shape)
classes = np.unique(train_labels)
nClasses = len(classes)
print('Total number of outputs: ', nClasses)
print('Output classes: ', classes)
plt.figure(figsize=[4, 2])
plt.subplot(121)
plt.imshow(train_images[0])
plt.title('Ground Truth: {}'.format(train_labels[0]))
plt.subplot(122)
plt.imshow(test_images[0])
plt.title('Ground Truth: {}'.format(test_labels[0]))
plt.show()

In [ ]:
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
from tensorflow.keras.utils import to_categorical
print('Original label: ', train_labels[0])
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print('After conversion to categorical (one-hot): ', train_labels[0])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=train_images.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nClasses, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, batch_size=256, epochs=25, verbose=1, validation_split=0.20)

In [ ]:
plt.figure(figsize=[8, 6])
plt.plot(history.history['loss'], 'r', linewidth=3.0)
plt.plot(history.history['val_loss'], 'b', linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'], fontsize=18)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Loss Curves', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=[8, 6])
plt.plot(history.history['accuracy'], 'r', linewidth=3.0)
plt.plot(history.history['val_accuracy'], 'b', linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.title('Accuracy Curves', fontsize=16)
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

In [ ]:
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

In [ ]:
model_json = model.to_json()
with open('/content/drive/My Drive/Colab Notebooks/cnn_model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('/content/drive/My Drive/Colab Notebooks/cnn_model.h5') 

In [ ]:
from tensorflow.keras.models import model_from_json
json_file = open('/content/drive/My Drive/Colab Notebooks/cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/content/drive/My Drive/Colab Notebooks/cnn_model.h5')

In [ ]:
loaded_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
test_loss, test_accuracy = loaded_model.evaluate(test_images, test_labels)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

In [ ]:
labels =  ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
plt.figure(figsize=(13, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.axis('off')
    plt.imshow(test_images[i])
    gt_string = f'Ground Truth: {labels[np.argmax(test_labels[i])]}'
    pred_string = f'Prediction: {labels[np.argmax(loaded_model.predict(test_images[i:i+1]))]}' 
    plt.title(f'{gt_string}\n{pred_string}')
    plt.tight_layout()
plt.show()